# Product Recommendation System.
In this notebook I am building two types of recommendation systems.

User-based recommendation system
Item-based recommendation system
One recommendation system is selected based upon the performance of both models.

In [3]:
print("Recommnedation System")

Recommnedation System


In [4]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns

# Reading the dataset

In [6]:
#df_org=pd.read_csv('/content/gdrive/MyDrive/MyCapstone (1)/data/sample30.csv')
df_org=pd.read_csv('data/sample30.csv')
df_org.shape

(30000, 15)

We require only 3 columns to build our recommendation system
1. id
2. reviews_username
3. reviews_rating

In [7]:
df = df_org[['id','reviews_username','reviews_rating']]

# Data Analysis

In [8]:
df.shape

(30000, 3)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                30000 non-null  object
 1   reviews_username  29937 non-null  object
 2   reviews_rating    30000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 703.2+ KB


In [10]:
df.head()

,id,reviews_username,reviews_rating
0,AV13O1A8GV-KLJ3akUyj,joshua,5
1,AV14LG0R-jtxr-f38QfS,dorothy w,5
2,AV14LG0R-jtxr-f38QfS,dorothy w,5
3,AV16khLE-jtxr-f38VFn,rebecca,1
4,AV16khLE-jtxr-f38VFn,walker557,1


In [11]:
df.id.nunique()

271

In [12]:
df.reviews_username.nunique()

24914

## Data Cleaning

In [13]:
# Dropping rows where username is missing
df_user = df.dropna()

In [14]:
df_user.shape

(29937, 3)

## Handling duplicate data

In [15]:
duplicate = df_user[df_user.duplicated(['reviews_username', 'id'])]
duplicate

,id,reviews_username,reviews_rating
2,AV14LG0R-jtxr-f38QfS,dorothy w,5
12,AV16khLE-jtxr-f38VFn,jds1992,1
44,AV1h6Gu0glJLPUi8IjA_,solo,4
79,AV1l8zRZvKc47QAVhnAv,mylifeinheels,3
90,AV1l8zRZvKc47QAVhnAv,mylifeinheels,3
...,...,...,...
29807,AVpfW8y_LJeJML437ySW,joycakes,5
29827,AVpfW8y_LJeJML437ySW,emily95,5
29841,AVpfW8y_LJeJML437ySW,brookieboo1994,5
29921,AVpfW8y_LJeJML437ySW,talmariejohn,5


In [16]:
duplicate = df_user[df_user.duplicated()]
duplicate

,id,reviews_username,reviews_rating
2,AV14LG0R-jtxr-f38QfS,dorothy w,5
12,AV16khLE-jtxr-f38VFn,jds1992,1
44,AV1h6Gu0glJLPUi8IjA_,solo,4
79,AV1l8zRZvKc47QAVhnAv,mylifeinheels,3
90,AV1l8zRZvKc47QAVhnAv,mylifeinheels,3
...,...,...,...
29807,AVpfW8y_LJeJML437ySW,joycakes,5
29827,AVpfW8y_LJeJML437ySW,emily95,5
29841,AVpfW8y_LJeJML437ySW,brookieboo1994,5
29921,AVpfW8y_LJeJML437ySW,talmariejohn,5


In [17]:
# Extract duplicate rows
#df_user[df_user[id=='AV1YGDqsGV-KLJ3adc-O'] and df_user[df_user[reviews_username=='laura']]
        
df_user[(df_user['id']=='AV1YGDqsGV-KLJ3adc-O') & (df_user['reviews_username']=='laura')]        

,id,reviews_username,reviews_rating
695,AV1YGDqsGV-KLJ3adc-O,laura,1
746,AV1YGDqsGV-KLJ3adc-O,laura,2


In [18]:
# dropping ALL duplicate values
df_user.drop_duplicates(keep = 'first', inplace = True)

<ipython-input-18-25316b5d7dec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user.drop_duplicates(keep = 'first', inplace = True)


In [19]:
df_user.shape

(27739, 3)

In [20]:
#Getting rows where same user provided different ratings for smae item.
duplicate = df_user[df_user.duplicated(['reviews_username', 'id'])]
duplicate

,id,reviews_username,reviews_rating
746,AV1YGDqsGV-KLJ3adc-O,laura,2
905,AV1YGDqsGV-KLJ3adc-O,suri,5
1705,AVpe31o71cnluZ0-YrSD,chris,5
1884,AVpe41TqilAPnD_xQH3d,steve,3
1922,AVpe41TqilAPnD_xQH3d,james,4
...,...,...,...
28150,AVpfRTh1ilAPnD_xYic2,dusty,5
28158,AVpfRTh1ilAPnD_xYic2,tony,5
28269,AVpfRTh1ilAPnD_xYic2,melbournesandy,5
28354,AVpfRTh1ilAPnD_xYic2,7.87E+11,5


In [21]:
#Taking mean of different ratings provided by user for same item
df2 = df_user.groupby(['id','reviews_username']).mean().reset_index()

In [22]:
df2[(df2['id']=='AV1YGDqsGV-KLJ3adc-O') & (df2['reviews_username']=='laura')]  

,id,reviews_username,reviews_rating
187,AV1YGDqsGV-KLJ3adc-O,laura,1.5


In [23]:
print(df2.shape)
df2.head()

(27588, 3)


,id,reviews_username,reviews_rating
0,AV13O1A8GV-KLJ3akUyj,joshua,5.0
1,AV14LG0R-jtxr-f38QfS,dorothy w,5.0
2,AV16khLE-jtxr-f38VFn,amanda,5.0
3,AV16khLE-jtxr-f38VFn,ashley a,5.0
4,AV16khLE-jtxr-f38VFn,beccagrl532,1.0


In [24]:
# After removing duplicates
print(df2.shape)
print("Unique users :",df2.reviews_username.nunique())
print("Unique items : ",df2.id.nunique())


(27588, 3)
Unique users : 24914
Unique items :  271


# Dividing the dataset into train and test

In [25]:
# Test and Train split of the dataset.
from sklearn.model_selection import train_test_split
train, test = train_test_split(df2, test_size=0.30, random_state=31)

In [26]:
print(train.shape)
print(test.shape)

(19311, 3)
(8277, 3)


In [27]:
# Pivot the train ratings' dataset into matrix format in which columns are items and the rows are user IDs.
df_pivot = train.pivot(
    index='reviews_username',
    columns='id',
    values='reviews_rating'
).fillna(0)

df_pivot.head(10)

id,AV13O1A8GV-KLJ3akUyj,AV14LG0R-jtxr-f38QfS,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YIch7GV-KLJ3addeG,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,...,AVpfrTyiLJeJML43BrSI,AVpfrfHF1cnluZ0-pRai,AVpfrgjFLJeJML43BvCc,AVpfs0tUilAPnD_xgqN2,AVpfsQoeilAPnD_xgfx5,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpftymALJeJML43CZ6y,AVpfv4TlilAPnD_xhjNS,AVpfvieo1cnluZ0-qdnu
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sab00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01impala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0325home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06stidriver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
08dallas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
09mommy11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10ten,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1143mom,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
print(df_pivot.shape)
print(train.reviews_username.nunique())
print(train.id.nunique())

(17872, 245)
17872
245


In [29]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()

In [30]:
# The items not rated by user is marked as 1 for prediction. 
dummy_train['reviews_rating'] = dummy_train['reviews_rating'].apply(lambda x: 0 if x>=1 else 1)

In [31]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot(
    index='reviews_username',
    columns='id',
    values='reviews_rating'
).fillna(1)

In [32]:
dummy_train.head()

id,AV13O1A8GV-KLJ3akUyj,AV14LG0R-jtxr-f38QfS,AV16khLE-jtxr-f38VFn,AV1YGDqsGV-KLJ3adc-O,AV1YIch7GV-KLJ3addeG,AV1YlENIglJLPUi8IHsX,AV1YmBrdGV-KLJ3adewb,AV1YmDL9vKc47QAVgr7_,AV1Ymf_rglJLPUi8II2v,AV1Yn94nvKc47QAVgtst,...,AVpfrTyiLJeJML43BrSI,AVpfrfHF1cnluZ0-pRai,AVpfrgjFLJeJML43BvCc,AVpfs0tUilAPnD_xgqN2,AVpfsQoeilAPnD_xgfx5,AVpfthSailAPnD_xg3ON,AVpftikC1cnluZ0-p31V,AVpftymALJeJML43CZ6y,AVpfv4TlilAPnD_xhjNS,AVpfvieo1cnluZ0-qdnu
reviews_username,,,,,,,,,,,,,,,,,,,,,
00dog3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00sab00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
01impala,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0325home,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
06stidriver,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# User Similarity Matrix

## Using Cosine Similarity


In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

In [ ]:
user_correlation.shape

## Using adjusted Cosine

In [ ]:
# Using adjusted Cosine

Here, we are not removing the NaN values and calculating the mean only for the items rated by the user

In [ ]:
# Create a user-item matrix.
df_pivot = train.pivot(
    index='reviews_username',
    columns='id',
    values='reviews_rating'
)

In [ ]:
df_pivot.head(10)

### Normalising the rating of the item for each user around 0 mean




In [ ]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [ ]:
df_subtracted.head()

### Finding cosine similarity

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

## Prediction - User User


In [ ]:
user_correlation[user_correlation<0]=0
user_correlation

In [ ]:
user_predicted_ratings = np.dot(user_correlation, df_pivot.fillna(0))
user_predicted_ratings

In [ ]:
user_predicted_ratings.shape

In [ ]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

In [ ]:
# Predicting the item ratings for some user.
user_input = "laura"
print(user_input)

In [ ]:
# Top 20 recommnedations
d = user_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
d

## Evaluation - User User

In [ ]:
# Find out the common users of test and train dataset.
common = test[test.reviews_username.isin(train.reviews_username)]
common.shape

In [ ]:
common.head()

In [ ]:
# convert into the user-item matrix.
common_user_based_matrix = common.pivot_table(index='reviews_username', columns='id', values='reviews_rating')

In [ ]:
# Convert the user_correlation matrix into dataframe.
user_correlation_df = pd.DataFrame(user_correlation)

In [ ]:
df_subtracted.head(1)

In [ ]:
user_correlation_df['userId'] = df_subtracted.index
user_correlation_df.set_index('userId',inplace=True)
user_correlation_df.head()

In [ ]:
common.head(1)

In [ ]:
list_name = common.reviews_username.tolist()

user_correlation_df.columns = df_subtracted.index.tolist()


user_correlation_df_1 =  user_correlation_df[user_correlation_df.index.isin(list_name)]

In [ ]:
user_correlation_df_1.shape

In [ ]:
user_correlation_df_2 = user_correlation_df_1.T[user_correlation_df_1.T.index.isin(list_name)]

In [ ]:
user_correlation_df_3 = user_correlation_df_2.T

In [ ]:
user_correlation_df_3.head()

In [ ]:
user_correlation_df_3.shape

In [ ]:
user_correlation_df_3[user_correlation_df_3<0]=0

common_user_predicted_ratings = np.dot(user_correlation_df_3, common_user_based_matrix.fillna(0))
common_user_predicted_ratings

In [ ]:
dummy_test = common.copy()

dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)

dummy_test = dummy_test.pivot_table(index='reviews_username', columns='id', values='reviews_rating').fillna(0)

In [ ]:
dummy_test.shape

In [ ]:
common_user_predicted_ratings = np.multiply(common_user_predicted_ratings,dummy_test)

In [ ]:
common_user_predicted_ratings.head(2)

Calculating the RMSE for only the items rated by user. For RMSE, normalising the rating to (1,5) range.




In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_user_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

In [ ]:
common_ = common.pivot_table(index='reviews_username', columns='id', values='reviews_rating')

In [ ]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [ ]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

# Using Item similarity


## Item Based Similarity

In [ ]:
df_pivot = train.pivot(
    index='reviews_username', columns='id', values='reviews_rating'
).T

df_pivot.head()

Normalising the item rating for each movie for using the Adujsted Cosine

In [ ]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [ ]:
df_subtracted.head()

Finding the cosine similarity using pairwise distances approach

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

Filtering the correlation only for which the value is greater than 0. (Positively correlated)

In [ ]:
item_correlation[item_correlation<0]=0
item_correlation

## Prediction - Item Item

In [ ]:
item_predicted_ratings = np.dot((df_pivot.fillna(0).T),item_correlation)
item_predicted_ratings

In [ ]:
item_predicted_ratings.shape

In [ ]:
dummy_train.shape

### Filtering the rating only for the items not rated by the user for recommendation


In [ ]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

### Finding the top 20 recommendation for the *user*



In [ ]:
# Take the user ID as input
user_input = "laura"
print(user_input)

In [ ]:
# Recommending the Top 20 products to the user.
d = item_final_rating.loc[user_input].sort_values(ascending=False)[0:20]
d

# Evaluation - Item Item

Evaluation will we same as I have done above for the prediction. The only difference being, I am evaluating for the item already rated by the user insead of predicting it for the item not rated by the user. 

In [ ]:
test.columns

In [ ]:
common =  test[test.id.isin(train.id)]
common.shape

In [ ]:
common.head(4)

In [ ]:
common_item_based_matrix = common.pivot_table(index='reviews_username', columns='id', values='reviews_rating').T

common_item_based_matrix.shape

In [ ]:
item_correlation_df = pd.DataFrame(item_correlation)

item_correlation_df.head()

In [ ]:
item_correlation_df['id'] = df_subtracted.index
item_correlation_df.set_index('id',inplace=True)
item_correlation_df.head()

In [ ]:

list_name = common.id.tolist()

In [ ]:
item_correlation_df.columns = df_subtracted.index.tolist()

item_correlation_df_1 =  item_correlation_df[item_correlation_df.index.isin(list_name)]

In [ ]:
item_correlation_df_2 = item_correlation_df_1.T[item_correlation_df_1.T.index.isin(list_name)]

item_correlation_df_3 = item_correlation_df_2.T

In [ ]:
item_correlation_df_3.head()

In [ ]:
item_correlation_df_3[item_correlation_df_3<0]=0

common_item_predicted_ratings = np.dot(item_correlation_df_3, common_item_based_matrix.fillna(0))
common_item_predicted_ratings


In [ ]:
common_item_predicted_ratings.shape

Dummy test will be used for evaluation. To evaluate, we will only make prediction on the items rated by the user. So, this is marked as 1. This is just opposite of dummy_train



In [ ]:
dummy_test = common.copy()

dummy_test['reviews_rating'] = dummy_test['reviews_rating'].apply(lambda x: 1 if x>=1 else 0)

dummy_test = dummy_test.pivot_table(index='reviews_username', columns='id', values='reviews_rating').T.fillna(0)

common_item_predicted_ratings = np.multiply(common_item_predicted_ratings,dummy_test)

The products not rated is marked as 0 for evaluation. And make the item- item matrix representaion.


In [ ]:
common_ = common.pivot_table(index='reviews_username', columns='id', values='reviews_rating').T

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_item_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

In [ ]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [ ]:
rmse1 = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse1)

# Comaparing both recommendation systems


In [ ]:
print("RMSE of User-User recommendation is :",rmse);
print("RMSE of Item-Item recommendation is :",rmse1);

RMSE of user-user is better then item-item

Let's check for how many users recommendation system will predict 0 rating for all items.

In [ ]:
def userCountFor0(selectedDf) :
    mySet = set()
    for user in train['reviews_username'].tolist() :
        res = selectedDf.loc[user].sort_values(ascending=False)[0:1]
        a = str(res.tolist())
        #print(a)
        if(a=='[0.0]') :
            mySet.add(user)

    return len(mySet)    

In [ ]:
userModel0Count = userCountFor0(user_final_rating)
print("User Count for which User-User recommendation system will predict 0 rating for all items : ",userModel0Count)
print("%age of user with 0 count ",round(((userModel0Count/train.reviews_username.nunique())*100),4))

In [ ]:
itemModel0Count = userCountFor0(item_final_rating)
print("User Count for which Item-Item recommendation system will predict 0 rating for all items : ",itemModel0Count)
print("%age of user with 0 count ",round(((itemModel0Count/train.reviews_username.nunique())*100),4))

We can see that % age of user, for whom recommendation system will predict 0 rating for all items is very very high in User based recommendation system.

So we are selecting the Item-Item based recommendation system for final deployment.